# Neural Network for Purchase Propensity

1. Generate **synthetic data** to simulate user behavior.
2. Train two models:
   - **Baseline Logistic Regression** (simple, interpretable)
   - **Neural Network (MLP)** (more complex, flexible)
3. Compare performance using clear metrics and visualizations.
4. Highlight **what leaders should focus on** in evaluation and decision‑making.

In [ ]:
# Setup
import numpy as np, pandas as pd
from numpy.random import default_rng
import matplotlib.pyplot as plt, warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss, roc_curve, precision_recall_curve
from sklearn.calibration import calibration_curve

rng = default_rng(0)
warnings.filterwarnings("ignore")


## Generate synthetic data
We simulate visitor behavior (sessions, pages, cart actions, channel, device, etc.) and whether they purchase in the next 48h (binary target).

In [ ]:
N = 1000
sessions = rng.poisson(2, N)
pages = rng.poisson(5, N)
cart_adds = rng.poisson(0.3*sessions)
device = rng.choice(['mobile','desktop'], N, p=[0.7,0.3])
country = rng.choice(['US','CA','GB'], N, p=[0.7,0.2,0.1])
channel = rng.choice(['direct','search','social','email'], N)

# Latent score for purchase probability
x = -3 + 0.2*pages + 0.4*cart_adds + (channel=='email')*0.5 + (device=='desktop')*0.2
p = 1/(1+np.exp(-x))
y = rng.binomial(1, p)

df = pd.DataFrame({
    'sessions': sessions,
    'pages': pages,
    'cart_adds': cart_adds,
    'device': device,
    'country': country,
    'channel': channel,
    'purchase_next_48h': y
})
df.head()

## Train/Test split
We hold out 20% of data for final evaluation.

In [ ]:
X = df.drop(columns=['purchase_next_48h'])
y = df['purchase_next_48h']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

num_cols = ['sessions','pages','cart_adds']
cat_cols = ['device','country','channel']

pre = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

## Baseline - Logistic Regression
Simple, interpretable, runs fast.

In [ ]:
logreg = Pipeline([
    ('pre', pre),
    ('clf', LogisticRegression(max_iter=300))
])
logreg.fit(X_train, y_train)
probs_lr = logreg.predict_proba(X_test)[:,1]

auc_lr = roc_auc_score(y_test, probs_lr)
pr_lr = average_precision_score(y_test, probs_lr)
brier_lr = brier_score_loss(y_test, probs_lr)

result = pd.DataFrame({
    "LR result":{
        "roc_auc_score": auc_lr,
        "average_precision_score": pr_lr,
        "brier_score_loss": brier_lr
    }
})

display(result.round(3))

## Neural Network (MLP)
A simple feedforward network (multi‑layer perceptron). More flexible but less interpretable.

In [ ]:
mlp = Pipeline([
    ('pre', pre),
    ('clf', MLPClassifier(
        hidden_layer_sizes=(64, 32, 16),   # 3 layers, more expressive
        activation='relu',                 # modern default
        solver='adam',                     # adaptive optimizer
        alpha=0.1,                         # L2 regularization
        learning_rate='adaptive',          # slows down as it converges
        max_iter=3000,                     # allow more epochs
        early_stopping=True,               # stops if no val improvement
        n_iter_no_change=50,               # patience for early stopping
        random_state=0
    ))
])
mlp.fit(X_train, y_train)
probs_nn = mlp.predict_proba(X_test)[:,1]

auc_nn = roc_auc_score(y_test, probs_nn)
pr_nn = average_precision_score(y_test, probs_nn)
brier_nn = brier_score_loss(y_test, probs_nn)

result = pd.DataFrame({
    "NN result":{
        "roc_auc_score": auc_nn,
        "average_precision_score": pr_nn,
        "brier_score_loss": brier_nn
    }
})

display(result.round(3))

# Training Loss curve
clf = mlp.named_steps["clf"]
plt.figure(figsize=(6,4))
plt.plot(clf.loss_curve_, marker="o")
plt.xlabel("epoch"); plt.ylabel("loss")
plt.title("MLP training loss curve")
plt.tight_layout(); plt.show()

## Baseline vs. Neural Net
- **ROC‑AUC** (ranking quality): How well the model ranks likely buyers above non-buyers. (higher=better)
- **PR‑AUC** (precision‑recall for rare positives): How well the model finds true buyers among the top predictions when buyers are rare. (higher=better)
- **Brier score** (calibration): How well the predicted probabilities match reality (calibration). (lower=better)

In [ ]:
results = pd.DataFrame({
    "Logistic Regression": {
        "ROC-AUC (higher=better)": auc_lr,
        "PR-AUC (higher=better)": pr_lr,
        "Brier (lower=better)": brier_lr
    },
    "Neural Net": {
        "ROC-AUC (higher=better)": auc_nn,
        "PR-AUC (higher=better)": pr_nn,
        "Brier (lower=better)": brier_nn
    }
})

display(results.T.round(3))

# ROC curves
fpr_lr, tpr_lr, _ = roc_curve(y_test, probs_lr)
fpr_nn, tpr_nn, _ = roc_curve(y_test, probs_nn)

plt.figure(figsize=(6,5))
plt.plot(fpr_lr, tpr_lr, label=f"Logistic (AUC={auc_lr:.3f})")
plt.plot(fpr_nn, tpr_nn, label=f"Neural Net (AUC={auc_nn:.3f})")
plt.plot([0,1],[0,1],'--', color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve Comparison")
plt.legend()
plt.show()

# Precision-Recall curves
prec_lr, rec_lr, _ = precision_recall_curve(y_test, probs_lr)
prec_nn, rec_nn, _ = precision_recall_curve(y_test, probs_nn)

plt.figure(figsize=(6,5))
plt.plot(rec_lr, prec_lr, label=f"Logistic (PR-AUC={pr_lr:.3f})")
plt.plot(rec_nn, prec_nn, label=f"Neural Net (PR-AUC={pr_nn:.3f})")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve Comparison")
plt.legend()
plt.show()

# Calibration (reliability) curve
prob_true_lr, prob_pred_lr = calibration_curve(y_test, probs_lr, n_bins=12, strategy='quantile')
prob_true_nn, prob_pred_nn = calibration_curve(y_test, probs_nn, n_bins=12, strategy='quantile')

plt.figure(figsize=(6,5))
plt.plot(prob_pred_lr, prob_true_lr, marker='o', label=f"Logistic (Brier={brier_lr:.3f})")
plt.plot(prob_pred_nn, prob_true_nn, marker='o', label=f"Neural Net (Brier={brier_nn:.3f})")
plt.plot([0,1],[0,1],'--')
plt.xlabel("Predicted probability (bin mean)")
plt.ylabel("Observed frequency")
plt.title("Calibration")
plt.legend()
plt.show()

## Decision Making
- **If baseline is close**: stick with logistic regression (simpler, transparent, easier to monitor).
- **If neural net shows clear lift**: consider adopting, but demand **extra checks**:
  - Calibration: are probabilities reliable?
  - Fairness: does performance differ by device/country?
  - Ops: can it run fast enough for homepage personalization?
- **Always tie back to business metrics**: incremental conversions, ROI, and fairness guardrails.


- The lesson: More complex is not always better. Leaders should weigh **performance vs. simplicity, interpretability, fairness, and operations**.